In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# dx = 1 km; Np = 1M; Nt = 5 min
data1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc', decode_timedelta=True) #***
parcel1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc', decode_timedelta=True) #***
res='1km';t_res='5min'
Np_str='1e6'

# # dx = 1km; Np = 50M; Nz = 95
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min_95nz'; Np_str='50e6'

# # dx = 250m; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_250m_1min_50M.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_250m_1min_50M.nc', decode_timedelta=True) #***
# res='250m'; t_res='1min'; Np_str='50e6'

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

#####

#Import StatisticalFunctions 
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import StatisticalFunctions
from StatisticalFunctions import * # import NumericalFunctions 

In [ ]:
###########################
#Plotting
plotting=False #KEEP FALSE WHEN JOB ARRAYS IS RUNNING
plotting=True

In [ ]:
#READING BACK IN
# import pickle
# dir2 = dir + f'Project_Algorithms/Domain_Profiles/'
# input_file = dir2 + f'mean_lfc_{res}_{t_res}_{Np_str}.pkl'

# with open(input_file, 'rb') as f:
#     mean_LFC = pickle.load(f)
# print(mean_LFC)

def LoadMeanLFC():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"MeanLFC_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        MeanLFC = pickle.load(f)
    return MeanLFC
MeanLFC=LoadMeanLFC()
mean_LFC=MeanLFC
print(f"Mean LFC is: {MeanLFC}\n")


def LoadAllCloudBase():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        all_cloudbase = pickle.load(f)
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
cloudbase=min_all_cloudbase
print(f"Minimum Cloudbase is: {cloudbase}\n")

In [ ]:
#thresholds
w_thresh1=0.1
w_thresh2=0.5
qcqi_thresh=1e-6
dir2=dir+'Project_Algorithms/Domain_Profiles/'

In [ ]:
def GetPropertyProfile(type):
    if type == "general":
        property_input_file = dir2 + f'OUTPUT/general_eulerian_profiles_{res}_{t_res}_{Np_str}.h5'
    elif type == "cloudy":
        property_input_file = dir2 + f'OUTPUT/cloudy_eulerian_profiles_{res}_{t_res}_{Np_str}.h5'

    output = {}
    with h5py.File(property_input_file, 'r') as f:
        for var in ['w', 'qv', 'th']:
            profile = f[f'profile_{var}'][:]
            try:
                profile_squares = f[f'profile_{var}_squares'][:]
                profile_SE = ProfileStandardError(profile, profile_squares)
                # profile_SE = ProfileStandardDeviation(profile, profile_squares)
            except:
                #IF SE NOT AVAILABLE
                dummy=profile.copy()
                dummy[:, 0] = np.zeros(len(dummy[:, 0]))
                dummy=dummy[dummy[:,1]>0]
                profile_SE=dummy
            
            output[f'profile_{var}'] = profile
            output[f'profile_{var}_SE'] = profile_SE


    return output

def GetBudgetProfile(var_type,type):
    if type=='general':
        input_file = dir2+f'OUTPUT/{var_type}_BUDGET_general_eulerian_profiles_{res}_{t_res}_{Np_str}.h5' 
    elif type=='cloudy':
        input_file = dir2+f'OUTPUT/{var_type}_BUDGET_cloudy_eulerian_profiles_{res}_{t_res}_{Np_str}.h5' 
    return input_file

def LoadProfiles(input_file, budget_type):
    with h5py.File(input_file, 'r') as f:
        keys_list=[]
        for key in f.keys():
            if budget_type in key:
                globals()[key] = np.array(f[key])
                if '_squares' not in key:
                    keys_list.append(key)
        for key in keys_list:
            try:
                globals()[f"{key}_SE"]=ProfileStandardError(globals()[f"{key}"],globals()[f"{key}_squares"])
                # globals()[f"{key}_SE"]=ProfileStandardDeviation(globals()[f"{key}"],globals()[f"{key}_squares"])

            except:
                #IF SE NOT AVAILABLE
                dummy=globals()[f"{key}"].copy()
                dummy[:, 0] = np.zeros(len(dummy[:, 0]))
                dummy=dummy[dummy[:,1]>0]
                globals()[f"{key}_SE"]=dummy

In [ ]:
if plotting==True:
    #WBUDGET
    label=1
    
    import h5py
    #Produced averaged profiles for plotting
    def averaged_profiles(var):
        profile=globals()[f"profile_{var}"]
    
        out_var=profile[ (profile[:, 1] > 1)]; #gets rid of rows that have no data
        out_var=np.array([out_var[:, 0] / out_var[:, 1], out_var[:, 2]]).T #divides the data column by the counter column
        return out_var
    
    ####################################################################
    dim='1km'
    # dim='250m'
    label=1
    
    #Uses Plotting Function 
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))
    plt.subplots_adjust(top=0.95)
    
    fig.suptitle(f"Average General (" + r"$w_{thresh}$: w>= " + f"{w_thresh1} & " + r"$(q_c+q_i)_{thresh}$ < " + f"{qcqi_thresh}" + ") vs Cloudy (" + r"$w_{thresh}$: w>= " + f"{w_thresh2} & " + r"$(q_c+q_i)_{thresh}$ >= " + f"{qcqi_thresh}" + ") Updraft WBUDGET Profiles (Eulerian)")
    ax1, ax2, ax3, ax4 = axs.flatten()
    
    ####################################################################
    for type in ["general","cloudy"]:
        print(f'currently on type {type}')
        
        property_profiles=GetPropertyProfile(type) #*#*
        for key, value in property_profiles.items():
            globals()[key] = value
        # input_file=GetBudgetProfile('WQVTH',type)
        input_file=GetBudgetProfile('W',type)
        budget_type='wb';LoadProfiles(input_file,budget_type)
    
        #Plotting function
        def plotting_func(out_var,axis,label,color):
            axis.plot(out_var[:,0],out_var[:,1],label=label,color=color)
            axis.grid(True)
        
        #Uses Averaged_Profiles Function
        vars = [
        'w',
        'wb_hadv',
        'wb_vadv',
        'wb_hidiff',
        'wb_vidiff',
        'wb_hturb',
        'wb_vturb',
        'wb_pgrad',
        'wb_buoy'
    ]
        colors = [
        'black',    
        'blue',     
        'orange',   
        'green',    
        'red',      
        'purple',   
        'brown',    
        'pink',     
        'grey',
        'cyan'
    ]
    
        for var in vars:
            globals()[f"out_{var}"]=averaged_profiles(var)
            # globals()[f"out_{var}"]=ProfileMean(globals()[f"profile_{var}"])
    
        for var in vars:
            out_profile=globals()[f"out_{var}"]
            minmax=(np.min(out_profile[:,0]),np.max(out_profile[:,0]))
    
        if type=='general': #*#*
            axis=ax1
            profile=out_w.copy();
            plotting_func(out_w,axis,label=f'{type} ' + vars[0],color=colors[0])

            profile_SE=profile_w_SE;switch=1;factor=1;#factor=1.96
            axis.fill_betweenx(profile[:, 1], profile[:, 0] - factor*profile_SE[:,0]*switch, profile[:, 0] + factor*profile_SE[:,0], color=colors[0], alpha=0.1)
            
            axis.set_ylim(bottom=0,top=20)
            axis.set_xlabel('w (m/s)')
            axis.legend()
            axis=ax3
        if type=='cloudy':
            axis=ax2
            profile=out_w.copy()
            plotting_func(out_w,axis,label=f'{type} ' + vars[0],color=colors[1]);factor=1;#factor=1.96
            
            profile_SE=profile_w_SE;switch=1
            axis.fill_betweenx(profile[:, 1], profile[:, 0] - factor*profile_SE[:,0]*switch, profile[:, 0] + factor*profile_SE[:,0], color=colors[1], alpha=0.1)
            
            axis.set_ylim(bottom=0,top=20)
            axis.set_xlabel('w (m/s)')
            axis.legend()
            axis=ax4
        for var,xlabel,color in zip(vars[1:],vars[1:],colors[1:]):
            profile=globals()[f"out_{var}"].copy()
            plotting_func(profile,axis,label=xlabel,color=color)
            
            profile_SE = globals()[f"profile_{var}_SE"];switch=1;factor=1;#factor=1.96
            axis.fill_betweenx(profile[:, 1], profile[:, 0] - factor*profile_SE[:,0]*switch, profile[:, 0] + factor*profile_SE[:,0], color=color, alpha=0.1)

            axis.set_ylim(bottom=0,top=20)
            axis.set_ylabel('z (km)');
            axis.set_xlabel('(m/s/s)')
            axis.legend(fontsize='small'); #add legend
    
    #FIXING X LIMITS
    fix_x_limits([ax1, ax2])
    fix_x_limits([ax3, ax4])
    
    #MEAN CLOUD BASE
    # cloudbase=LoadCloudBase()
    axline_lw=1.2
    for axis in [ax1,ax2,ax3,ax4]:
        axis.axhline(cloudbase,color='purple',linestyle='dashed',lw=axline_lw)
        axis.axhline(mean_LFC/1000,color='green',linestyle='dashed',lw=axline_lw)
    
    #save plot
    # plt.savefig(dir+f'Project_Algorithms/plots/eulerian_WBudget_{res}_{t_res}_{Np_str}.jpg', bbox_inches='tight', dpi=300) 

In [ ]:
if plotting==True:
    #QV BUDGETS
    from matplotlib.ticker import ScalarFormatter
    label=1
    
    import h5py
    #Produced averaged profiles for plotting
    def averaged_profiles(var):
        profile=globals()[f"profile_{var}"]
    
        out_var=profile[ (profile[:, 1] > 1)]; #gets rid of rows that have no data
        out_var=np.array([out_var[:, 0] / out_var[:, 1], out_var[:, 2]]).T #divides the data column by the counter column
        return out_var
    
    ####################################################################
    dim='1km'
    label=1
    
    #Uses Plotting Function 
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))
    plt.subplots_adjust(top=0.95)
    
    fig.suptitle(f"Average General (" + r"$w_{thresh}$: w>= " + f"{w_thresh1} & " + r"$(q_c+q_i)_{thresh}$ < " + f"{qcqi_thresh}" + ") vs Cloudy (" + r"$w_{thresh}$: w>= " + f"{w_thresh2} & " + r"$(q_c+q_i)_{thresh}$ >= " + f"{qcqi_thresh}" + ") Updraft QVBUDGET Profiles (Eulerian)")
    ax1, ax2, ax3, ax4 = axs.flatten()
    
    ####################################################################
    for type in ["general","cloudy"]:
        print(f'currently on type {type}')
        
        property_profiles=GetPropertyProfile(type)
        for key, value in property_profiles.items():
            globals()[key] = value
        # input_file=GetBudgetProfile('WQVTH',type)
        input_file=GetBudgetProfile('QV',type)
        budget_type='qvb';LoadProfiles(input_file,budget_type)
    
        #Plotting function
        def plotting_func(out_var,axis,label,color):
            axis.plot(out_var[:,0],out_var[:,1],label=label,color=color)
            axis.grid(True)
        
        #Uses Averaged_Profiles Function
        vars = [
            'qv',
            'qvb_hadv',
            'qvb_vadv',
            'qvb_hidiff',
            'qvb_vidiff',
            'qvb_hturb',
            'qvb_vturb',
            'qvb_mp'
        ]
        
        colors = [
        'black',    
        'blue',     
        'orange',   
        'green',    
        'red',      
        'purple',   
        'brown',    
        'pink',     
        'grey',
        'cyan'
    ]
    
        for var in vars:
            globals()[f"out_{var}"]=averaged_profiles(var)
            globals()[f"out_{var}"][:,0]*=1000
            globals()[f"profile_{var}_SE"][:,0]*=1000
    
        if type=='general':
            axis=ax1
            profile=out_qv.copy()
            plotting_func(out_qv,axis,label=f'{type} ' + vars[0],color=colors[0])

            profile_SE=profile_qv_SE;switch=1;factor=1;#factor=1.96
            axis.fill_betweenx(profile[:, 1], profile[:, 0] - factor*profile_SE[:,0]*switch, profile[:, 0] + factor*profile_SE[:,0], color=colors[0], alpha=0.1)
            
            axis.set_ylim(bottom=0)
            axis.set_xlabel('qv (g/kg)')
            axis.legend()
            axis=ax3
        if type=='cloudy':
            axis=ax2
            profile=out_qv.copy()
            plotting_func(out_qv,axis,label=f'{type} ' + vars[0],color=colors[1])

            profile_SE=profile_qv_SE;switch=1;factor=1;#factor=1.96
            axis.fill_betweenx(profile[:, 1], profile[:, 0] - factor*profile_SE[:,0]*switch, profile[:, 0] + factor*profile_SE[:,0], color=colors[1], alpha=0.1)
            
            axis.set_ylim(bottom=0)
            axis.set_xlabel('qv (g/kg)')
            axis.legend()
            axis=ax4
        for var,xlabel,color in zip(vars[1:],vars[1:],colors[1:]):
            profile=globals()[f"out_{var}"].copy()
            plotting_func(profile,axis,label=xlabel,color=color)

            profile_SE = globals()[f"profile_{var}_SE"];switch=1;factor=1;#factor=1.96
            axis.fill_betweenx(profile[:, 1], profile[:, 0] - factor*profile_SE[:,0]*switch, profile[:, 0] + factor*profile_SE[:,0], color=color, alpha=0.1)
            
            axis.set_ylabel('z (km)');
            axis.set_xlabel('(g/kg/s)')
            axis.legend(fontsize='small'); #add legend
            axis.xaxis.set_major_formatter(ScalarFormatter())
            axis.xaxis.get_major_formatter().set_powerlimits((-1, 1))  # Set limits for scientific notation
    
    #FIXING X LIMITS
    fix_x_limits([ax1, ax2])
    fix_x_limits([ax3, ax4])
    
    #MEAN CLOUD BASE
    # cloudbase=LoadCloudBase()
    axline_lw=1.2
    for axis in [ax1,ax2,ax3,ax4]:
        axis.axhline(cloudbase,color='purple',linestyle='dashed',lw=axline_lw)
        axis.axhline(mean_LFC/1000,color='green',linestyle='dashed',lw=axline_lw)
    
                
    #save plot
    # plt.savefig(dir+f'Project_Algorithms/plots/eulerian_QVBudget_{res}_{t_res}_{Np_str}.jpg', bbox_inches='tight', dpi=300) 

In [ ]:
if plotting==True:
    #TH BUDGETS
    from matplotlib.ticker import ScalarFormatter
    label=1
    
    import h5py
    #Produced averaged profiles for plotting
    def averaged_profiles(var):
        profile=globals()[f"profile_{var}"]
    
        out_var=profile[ (profile[:, 1] > 1)]; #gets rid of rows that have no data
        out_var=np.array([out_var[:, 0] / out_var[:, 1], out_var[:, 2]]).T #divides the data column by the counter column
        return out_var
    
    ####################################################################
    dim='1km'
    label=1
    
    #Uses Plotting Function 
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))
    plt.subplots_adjust(top=0.95)
    
    fig.suptitle(f"Average General (" + r"$w_{thresh}$: w>= " + f"{w_thresh1} & " + r"$(q_c+q_i)_{thresh}$ < " + f"{qcqi_thresh}" + ") vs Cloudy (" + r"$w_{thresh}$: w>= " + f"{w_thresh2} & " + r"$(q_c+q_i)_{thresh}$ >= " + f"{qcqi_thresh}" + ") Updraft THBUDGET Profiles (Eulerian)")
    ax1, ax2, ax3, ax4 = axs.flatten()
    
    ####################################################################
    for type in ["general","cloudy"]:
        print(f'currently on type {type}')
        
        property_profiles=GetPropertyProfile(type)
        for key, value in property_profiles.items():
            globals()[key] = value
        # input_file=GetBudgetProfile('WQVTH',type)
        input_file=GetBudgetProfile('TH',type)
        budget_type='ptb';LoadProfiles(input_file,budget_type)
    
        #Plotting function
        def plotting_func(out_var,axis,label,color):
            axis.plot(out_var[:,0],out_var[:,1],label=label,color=color)
            axis.grid(True)
        
        #Uses Averaged_Profiles Function
        vars = [
            'th',
            'ptb_hadv',
            'ptb_vadv',
            'ptb_hidiff',
            'ptb_vidiff',
            'ptb_hturb',
            'ptb_vturb',
            'ptb_mp',
            'ptb_rad',
            'ptb_div',
            'ptb_diss'
        ]
        
        colors = [
        'black',    
        'blue',     
        'orange',   
        'green',    
        'red',      
        'purple',   
        'brown',    
        'pink',     
        'grey',
        'cyan'
    ]
    
        for var in vars:
            globals()[f"out_{var}"]=averaged_profiles(var)
    
        if type=='general':
            axis=ax1
            profile=out_th.copy()
            plotting_func(out_th,axis,label=f'{type} ' + vars[0],color=colors[0])

            profile_SE=profile_th_SE;switch=0;factor=1;#factor=1.96
            axis.fill_betweenx(profile[:, 1], profile[:, 0] - factor*profile_SE[:,0]*switch, profile[:, 0] + factor*profile_SE[:,0], color=colors[0], alpha=0.1)
            
            axis.set_ylim(bottom=0)
            axis.set_xlabel('th (K)')
            axis.legend()
            axis=ax3
        if type=='cloudy':
            axis=ax2
            profile=out_th.copy()
            plotting_func(out_th,axis,label=f'{type} ' + vars[0],color=colors[1])

            profile_SE=profile_th_SE;switch=0;factor=1;#factor=1.96
            axis.fill_betweenx(profile[:, 1], profile[:, 0] - factor*profile_SE[:,0]*switch, profile[:, 0] + factor*profile_SE[:,0], color=colors[1], alpha=0.1)
            
            axis.set_ylim(bottom=0)
            axis.set_xlabel('th (K)')
            axis.legend()
            axis=ax4
        for var,xlabel,color in zip(vars[1:],vars[1:],colors[1:]):
            profile = globals()[f"out_{var}"].copy()
            plotting_func(profile,axis,label=xlabel,color=color)

            profile_SE = globals()[f"profile_{var}_SE"];switch=1;factor=1;#factor=1.96
            axis.fill_betweenx(profile[:, 1], profile[:, 0] - factor*profile_SE[:,0]*switch, profile[:, 0] + factor*profile_SE[:,0], color=color, alpha=0.1)
            
            axis.set_ylabel('z (km)');
            axis.set_xlabel('(K/s)')
            axis.legend(fontsize='small'); #add legend
            axis.xaxis.set_major_formatter(ScalarFormatter())
            axis.xaxis.get_major_formatter().set_powerlimits((-1, 1))  # Set limits for scientific notation
    
    #FIXING X LIMITS
    fix_x_limits([ax1, ax2])
    fix_x_limits([ax3, ax4])
    
    #MEAN CLOUD BASE
    # cloudbase=LoadCloudBase()
    axline_lw=1.2
    for axis in [ax1,ax2,ax3,ax4]:
        axis.axhline(cloudbase,color='purple',linestyle='dashed',lw=axline_lw)
        axis.axhline(mean_LFC/1000,color='green',linestyle='dashed',lw=axline_lw)
    
                
    #save plot
    # plt.savefig(dir+f'Project_Algorithms/plots/eulerian_THBudget_{res}_{t_res}_{Np_str}.jpg', bbox_inches='tight', dpi=300) 
    #limited